In [1]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.storage.storage_context import StorageContext
from llama_index.llm_predictor import HuggingFaceLLMPredictor
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from IPython.display import Markdown, display
from llama_index import ServiceContext, LangchainEmbedding
import os
import pandas as pd

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
data_dir = "/home/jupyter/topic_articles"
persist_dir = "/home/jupyter/topic_indices"

In [4]:
! rm -rf /home/jupyter/topic_articles
! rm -rf /home/jupyter/topic_indices

In [5]:
article_df = pd.read_parquet("gs://scraped-news-article-data-null/2023-topics.parquet")
article_df.head()

,source,id,category,title,published,body,summary,summary_type,topic,probability
0,reuters,29667,Europe,European shares rise on upbeat cues from Powel...,2023-02-08T17:11:00,Feb 8 (Reuters) - European shares rose on Wedn...,* \n* STOXX 600 pulls back from 9-mth high to ...,BULLETS,-1,0.000000
1,reuters,79619,China,China Evergrande debt restructuring incentive ...,2023-04-27T06:05:00,"HONG KONG, April 27 (Reuters) - Embattled prop...",,NULL,310,0.281727
2,reuters,48010,European Markets,European stocks rally as banking worries fade,2023-03-30T16:27:00,March 30 (Reuters) - European stocks rose to n...,* \n* H&M posts surprise profit in Dec-Feb\n* ...,BULLETS,77,1.000000
3,reuters,67859,Aerospace & Defense,China gears up to compete with SpaceX's Starli...,2023-03-02T10:29:00,"BEIJING, March 2 (Reuters) - China's military-...",,NULL,-1,0.000000
4,reuters,109054,World,Russia's war on Ukraine latest: Moscow denies ...,2023-05-12T05:13:00,May 11 (Reuters) - Russia's defence ministry o...,,NULL,719,1.000000


In [6]:
from datetime import datetime


article_df["month"] = article_df.published.apply(lambda t: datetime.fromisoformat(t).month)
article_df.head()

,source,id,category,title,published,body,summary,summary_type,topic,probability,month
0,reuters,29667,Europe,European shares rise on upbeat cues from Powel...,2023-02-08T17:11:00,Feb 8 (Reuters) - European shares rose on Wedn...,* \n* STOXX 600 pulls back from 9-mth high to ...,BULLETS,-1,0.000000,2
1,reuters,79619,China,China Evergrande debt restructuring incentive ...,2023-04-27T06:05:00,"HONG KONG, April 27 (Reuters) - Embattled prop...",,NULL,310,0.281727,4
2,reuters,48010,European Markets,European stocks rally as banking worries fade,2023-03-30T16:27:00,March 30 (Reuters) - European stocks rose to n...,* \n* H&M posts surprise profit in Dec-Feb\n* ...,BULLETS,77,1.000000,3
3,reuters,67859,Aerospace & Defense,China gears up to compete with SpaceX's Starli...,2023-03-02T10:29:00,"BEIJING, March 2 (Reuters) - China's military-...",,NULL,-1,0.000000,3
4,reuters,109054,World,Russia's war on Ukraine latest: Moscow denies ...,2023-05-12T05:13:00,May 11 (Reuters) - Russia's defence ministry o...,,NULL,719,1.000000,5


In [7]:
month_filter = 4
article_df = article_df.loc[article_df.month == month_filter]
len(article_df.index)

14837

In [8]:
from pathlib import Path


topics = list(set(article_df.topic.unique()) - {-1})
top_sample = 300
for t in topics:
    topic_path = Path(data_dir, str(t))
    topic_path.mkdir(parents=True, exist_ok=True)
    topic_article_df = article_df.loc[article_df.topic == t]
    topic_article_df = topic_article_df.sort_values(by="probability", ascending=False)
    
    for i in range(len(topic_article_df.index)):
        doc_path = Path(data_dir, str(t), str(i) + ".txt")
        with open(doc_path, "w") as fp:
            fp.write(topic_article_df["body"].iloc[i])


In [9]:
with open("/home/jupyter/apikey", "r") as api_fp:
    api_key = api_fp.read().strip()

In [10]:
import os


os.environ["OPENAI_API_KEY"] = api_key.strip()

In [11]:
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing import cpu_count
import multiprocessing
import torch
from llama_index.prompts.prompts import SimpleInputPrompt


def build_topic_idx(t):
    topic_data_dir = Path(data_dir, str(t))
    topic_index_dir = Path(persist_dir, str(t))
        
    embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
    service_context = ServiceContext.from_defaults(chunk_size_limit=512, embed_model=embed_model)
    storage_context = StorageContext.from_defaults()
    documents = SimpleDirectoryReader(topic_data_dir).load_data()
        
    new_index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)
    storage_context.persist(persist_dir=topic_index_dir)


with tqdm(total=len(topics)) as progress:
    
    with Pool(processes=cpu_count() - 1) as pool:
        for _ in pool.imap_unordered(build_topic_idx, list(topics)):
            progress.update(1)
        
print("Finished building index")

  0%|          | 0/1312 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda
Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

100%|█████████▉| 1311/1312 [07:18<00:00,  5.06it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1861 tokens
> [build_index_from_nodes] Total embedding token usage: 1861 tokens


100%|██████████| 1312/1312 [07:19<00:00,  2.99it/s]

Finished building index
